In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import rcParams
import os
from collections import Counter
os.chdir("/home/wyh/scdata/")

In [2]:
sc.settings.set_figure_params(figsize=(3, 3))
sc.settings.set_figure_params(dpi=800, frameon=False, figsize=(3, 3), facecolor='white') 
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

# porcessing of original data

In [38]:
adata = sc.read("./combined_data/h5ad_file/combine_final0609.h5ad")
## preprocessing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

## regress cycling genes
cell_cycle_genes = pd.read_csv("./combined_data/cell_cycling_genes.csv",index_col=0)
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
s_genes = [x for x in s_genes.iloc[:,0] if x in adata.var_names]
g2m_genes = [x for x in g2m_genes.iloc[:,0] if x in adata.var_names]

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.regress_out(adata, ['nCount_RNA', 'nFeature_RNA',"S_score","G2M_score"])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
#sc.pl.pca(adata, color='CST3')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata)
adata_original.write("./combined_data/h5ad_file/combine_final0609.h5ad")

### Find markers of each cell types

In [29]:
import matplotlib as mpl
import pandas as pd

In [5]:
adata_original_new = sc.read_h5ad("./combined_data/h5ad_file/combine_final0609.h5ad")

In [159]:
def get_class_num(df,n):
    def get_num(x):
        if(len(x)>n):
            sampled_x = x.sample(n).reset_index(drop=True)
        else:
            sampled_x = x
        return(sampled_x)
            
    weighted_sample = df.groupby('Cell types').apply(get_num)  
    return(weighted_sample) 

            
df = adata_original_new.obs
df['barcode'] = df.index
weighted_sample = get_class_num(df,5000)

In [161]:
# 将细胞进行采样
adata_original_new_sample = adata_original_new[weighted_sample['barcodes'],].copy()
adata_original_new_sample.write("./adata_original_new_sample.h5ad")

In [3]:
adata_original_new_sample = sc.read_h5ad("./adata_original_new_sample.h5ad")
adata_original_new_sample.X = adata_original_new_sample.layers['counts']
Counter(adata.obs['Cell types'])

In [34]:
# 计算一下markers的信息
adata_original_new_sample.uns['log1p']['base'] = None
sc.tl.rank_genes_groups(adata_original_new_sample, 'Cell types', method='wilcoxon')
markers = sc.get.rank_genes_groups_df(adata_original_new_sample,group = None)
markers = markers.loc[markers.pvals<0.01,:]
markers.to_csv("/home/wyh/scdata/combined_data/Epithelial/submit/celltype_markers.csv")

In [36]:
result = adata_original_new_sample.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(10)

,B cells_n,B cells_p,Dendritic cells_n,Dendritic cells_p,Endothelial cells_n,Endothelial cells_p,Epithelial cells_n,Epithelial cells_p,Fibroblasts_n,Fibroblasts_p,...,Monocytes & Macrophages_n,Monocytes & Macrophages_p,NET_n,NET_p,NK & T cells_n,NK & T cells_p,Neutrophils_n,Neutrophils_p,Plasma cells_n,Plasma cells_p
0,CD37,0.0,HLA-DPB1,0.0,IGFBP7,0.0,KRT19,0.0,COL3A1,0.0,...,FTL,0.0,STMN1,0.0,IL32,0.0,CXCL8,0.0,MZB1,0.0
1,MS4A1,0.0,HLA-DPA1,0.0,HSPG2,0.0,KRT8,0.0,COL1A2,0.0,...,CTSB,0.0,PEG10,0.0,CD3D,0.0,NAMPT,0.0,SSR4,0.0
2,FAU,0.0,HLA-DRA,0.0,GNG11,0.0,KRT18,0.0,IGFBP7,0.0,...,IFI30,0.0,HMGN2,0.0,CD2,0.0,G0S2,0.0,DERL3,0.0
3,CXCR4,0.0,CD74,0.0,RAMP2,0.0,C19orf33,0.0,CALD1,0.0,...,TYROBP,0.0,SOX4,0.0,B2M,0.0,FTH1,0.0,FKBP11,0.0
4,HLA-DRA,0.0,HLA-DRB1,0.0,EGFL7,0.0,ELF3,0.0,COL1A1,0.0,...,CD68,0.0,HES6,0.0,TRBC2,0.0,SOD2,0.0,JCHAIN,0.0
5,CD79A,0.0,HLA-DQB1,0.0,PLVAP,0.0,AGR2,0.0,COL6A2,0.0,...,CD14,0.0,CKB,0.0,TRAC,0.0,S100A9,0.0,IGKC,0.0
6,CD74,0.0,HLA-DQA1,0.0,SPARCL1,0.0,LGALS4,0.0,TIMP1,0.0,...,AIF1,0.0,UCHL1,0.0,PTPRC,0.0,NEAT1,0.0,XBP1,0.0
7,EEF1B2,0.0,CST3,0.0,PECAM1,0.0,EPCAM,0.0,SELENOM,0.0,...,PSAP,0.0,SUMO2,0.0,CD3E,0.0,BCL2A1,0.0,IGLC1,0.0
8,LTB,0.0,TMSB10,0.0,A2M,0.0,CLDN4,0.0,BGN,0.0,...,FCER1G,0.0,PTMS,0.0,CCL5,0.0,IFITM2,0.0,IGHG1,0.0
9,HLA-DRB1,0.0,HLA-DMA,0.0,CALCRL,0.0,S100A6,0.0,C11orf96,0.0,...,LYZ,0.0,TUBA1A,0.0,HCST,0.0,SRGN,0.0,SEC11C,0.0


### Find markers of each subtype that are significantly up-regulated 

In [3]:
adata_original_new_final_filter = sc.read_h5ad("./combined_data/h5ad_file/combine_final0609.h5ad")
adata_original_new_final_filter.uns['log1p']['base'] = None

In [34]:
for itype in ["N_C0_N0", "N_C1_N2", "N_C2_N1", "N_C7_N0", "EC_C0_ACKR1", "EC_C5_PROX1",
              'F_C1_CFD','Per_C1_MYH11','Per_C3_STEAP4']:
    sc.tl.rank_genes_groups(adata_original_new_final_filter, groupby = 'subtype', method='wilcoxon',groups=[itype]) 
    result = adata_original_new_final_filter.uns['rank_genes_groups']
    groups = result['names'].dtype.names
    markers = pd.DataFrame(
        {group + '_' + key[:1]: result[key][group]
        for group in groups for key in ['names', 'pvals']})

    markers.to_csv("./combined_data/meta_data/"+itype+"_markers.csv")